In [42]:
import numpy as np
import random
import gurobipy as gp
import time

Variables to be used to generate nodes and cluster

In [43]:
np.random.seed(42)

# Number of nodes
n = 200
# Number of clusters
p = 2

capacity_mean = 10
capacity_stddev = 2
weight_mean = 1
weight_stddev = 0.1
lambda_param = 0.5

Generate Random Instances using capacity mean and standard deviation

In [44]:
def generate_instances(n, p, capacity_mean, capacity_stddev):
    # Generate 2D positions for n nodes
    nodes = np.random.rand(n, 2)

    # Generate capacities for clusters
    capacities = np.random.normal(capacity_mean, capacity_stddev, p)

    return nodes, capacities

Generate Random Weights using weight mean and standard deviation

In [45]:
def generate_weights(n, weight_mean, weight_stddev):
    # Generate weights for n nodes
    weights = np.random.normal(weight_mean, weight_stddev, n)

    return weights

 Euclidean distance between two points p and q

In [46]:
def euclidean_distance(p, q):
    return np.sqrt(np.sum((p-q)**2))

In [47]:
def solve_ccp(nodes, capacities, weights, lambda_param):
    start = time.time()
    n = nodes.shape[0]
    p = len(capacities)

    # Create gurobi model
    model = gp.Model('ccp')

    # Create decision variables
    x = {}
    y = {}

    # Update decision variables as mentioned in the problem statement
    for i in range(n):
        for j in range(p):
            x[i, j] = model.addVar(vtype=gp.GRB.BINARY, name=f'x[{i},{j}]')
        y[i] = model.addVar(vtype=gp.GRB.BINARY, name=f'y[{i}]')

    # Set objective function
    obj = gp.quicksum(euclidean_distance(nodes[i], nodes[j]) * x[i,j] for i in range(n) for j in range(p))
    obj += lambda_param * gp.quicksum(capacities[j]*y[j] for j in range(p))
    obj -= lambda_param * gp.quicksum(weights[i]*x[i,j] for i in range(n) for j in range(p))
    
    model.setObjective(obj, gp.GRB.MINIMIZE)

    # Add constraints
    for i in range(n):
        model.addConstr(gp.quicksum(x[i,j] for j in range(p)) == 1, name=f'assign[{i}]')

    model.addConstr(gp.quicksum(y[j] for j in range(p)) <= p, name='num_clusters')

    for i in range(n):
        for j in range(p):
            model.addConstr(x[i,j] <= y[j], name=f'x_c[{i},{j}]')

    # Solve model
    model.optimize()
    # Extract solution
    clusters = []
    for j in range(p):
        cluster = [i for i in range(n) if x[i,j].X > 0.5]
        clusters.append(cluster)
    end = time.time()
    return clusters, end-start

In [48]:
def jaccard_similarity(clusters_1, clusters_2):
    jaccard_scores = []
    for cluster_1 in clusters_1:
        jaccard_scores_per_cluster = []
        for cluster_2 in clusters_2:
            union = len(set(cluster_1).union(cluster_2))
            intersection = len(set(cluster_1).intersection(cluster_2))
            jaccard_index = intersection / union
            jaccard_scores_per_cluster.append(jaccard_index)
        jaccard_scores.append(max(jaccard_scores_per_cluster))
    return np.mean(jaccard_scores)

In [49]:
def rand_index(clusters_1, clusters_2):
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    n = len(clusters_1)
    for i in range(n):
        for j in range(i+1, n):
            if (clusters_1[i] == clusters_1[j] and clusters_2[i] == clusters_2[j]):
                tp += 1
            elif (clusters_1[i] != clusters_1[j] and clusters_2[i] != clusters_2[j]):
                tn += 1
            elif (clusters_1[i] == clusters_1[j] and clusters_2[i] != clusters_2[j]):
                fp += 1
            else:
                fn += 1

    return (tp + tn) / (tp + tn + fp + fn)


In [50]:
def insample_stability(clusters_reference,nodes, capacities, weights, lambda_param, num_runs=100):
    jaccard_scores = []
    for i in range(num_runs):
        clusters, _ = solve_ccp(nodes, capacities, weights, lambda_param)
        jaccard_scores.append(jaccard_similarity(clusters, clusters_reference))
    return np.mean(jaccard_scores)


In [51]:
def split_data(nodes, weights, train_ratio=0.7):
    n = nodes.shape[0]
    indices = np.arange(n)
    np.random.shuffle(indices)
    train_indices = indices[:int(n * train_ratio)]
    val_indices = indices[int(n * train_ratio):]
    return nodes[train_indices], weights[train_indices], nodes[val_indices], weights[val_indices]


In [52]:
def out_sample_stability(num_runs=10):
    rand_index_scores = []
    
    for i in range(num_runs):

        nodes, capacities = generate_instances(n, p, capacity_mean, capacity_stddev)
        weights = generate_weights(n, weight_mean, weight_stddev)
        
        nodes_train, weights_train, nodes_val, weights_val = split_data(nodes, weights)
        
        clusters_train,_ = solve_ccp(nodes_train, capacities, weights_train, lambda_param)
        clusters_val,_ = solve_ccp(nodes_val, capacities, weights_val, lambda_param)
        rand_index_scores.append(rand_index(clusters_train, clusters_val))
    return np.mean(rand_index_scores)

In [53]:
def greedy_cluster_nodes(nodes, capacities, weights):
    start = time.time()
    n = nodes.shape[0]
    p = len(capacities)

    # Create a list to store the clusters
    clusters = [[] for _ in range(p)]

    # Create a list to store the remaining capacities of each cluster
    remaining_capacities = capacities.copy()

    # Sort the nodes by their weights in descending order
    node_weights = [(node, weight) for node, weight in enumerate(weights)]
    node_weights.sort(key=lambda x: x[1], reverse=True)

    # Greedily assign each node to a cluster with enough capacity
    for node, weight in node_weights:
        for j in range(p):
            if remaining_capacities[j] >= weight:
                clusters[j].append(node)
                remaining_capacities[j] -= weight
                break
    end = time.time()
    # Return the list of clusters, each represented by a list of node indices
    return clusters, end - start


In [54]:
# Generate 2D nodes and random capacities for clusters
nodes, capacities = generate_instances(n, p, capacity_mean, capacity_stddev)
# Generate random weight for each node
weights = generate_weights(n, weight_mean, weight_stddev)

clusters, time1 = solve_ccp(nodes, capacities, weights, lambda_param)

greedy_clusters, time2 = greedy_cluster_nodes(nodes, capacities, weights)


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 601 rows, 600 columns and 1202 nonzeros
Model fingerprint: 0xe878e5bd
Variable types: 0 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-04, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 15.1698562
Presolve removed 601 rows and 600 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: -12.3219 15.1699 
No other solutions better than -12.3219

Optimal solution found (tolerance 1.00e-04)
Best objective -1.232188968768e+01, best bound -1.232188968768e+01, gap 0.0000%


In [61]:
def distance(p, q):
    return np.sqrt(np.sum((p-q)**2))

def find_VSS(n, p, capacity_mean, capacity_stddev, weight_mean, weight_stddev, lambda_param, num_simulations):
# Find the EEVS by generating multiple instances and solving the CCP for each instance
    EEVS = 0
    for i in range(num_simulations):
        nodes, capacities = generate_instances(n, p, capacity_mean, capacity_stddev)
        weights = generate_weights(n, weight_mean, weight_stddev)

        clusters, _ = solve_ccp(nodes, capacities, weights, lambda_param)
        obj_value = 0
        for cluster in clusters:
            for i in cluster:
                obj_value += distance(nodes[i], nodes[cluster[0]]) * weights[i]
            obj_value += capacities[clusters.index(cluster)] * lambda_param
        EEVS += obj_value
        
    EEVS = EEVS / num_simulations


        # Solve RP to find optimal objective value
    model = gp.Model('RP')
    
    # Create decision variables
    x = {}
    y = {}
    for i in range(n):
        for j in range(p):
            x[i, j] = model.addVar(vtype=gp.GRB.BINARY, name=f'x[{i},{j}]')
        y[i] = model.addVar(vtype=gp.GRB.BINARY, name=f'y[{i}]')
    
    # Set objective function
    obj = gp.quicksum(distance(nodes[i], nodes[j]) * x[i,j] for i in range(n) for j in range(p))
    obj += lambda_param * gp.quicksum(capacities[j]*y[j] for j in range(p))
    obj -= lambda_param * gp.quicksum(weights[i]*x[i,j] for i in range(n) for j in range(p))
    model.setObjective(obj, gp.GRB.MINIMIZE)
    
    # Add constraints
    for i in range(n):
        model.addConstr(gp.quicksum(x[i,j] for j in range(p)) == 1, name=f'assign[{i}]')
    
    model.addConstr(gp.quicksum(y[j] for j in range(p)) <= p, name='num_clusters')
    
    for i in range(n):
        for j in range(p):
            model.addConstr(x[i,j] <= y[j], name=f'x_c[{i},{j}]')
    
    # Solve model
    model.optimize()
    RP = model.ObjVal

    

    # Calculate the VSS
    VSS = EEVS - RP

    return VSS


In [62]:
num_simulations = 100
np.random.seed(42)

# Number of nodes
n = 200
# Number of clusters
p = 2

capacity_mean = 10
capacity_stddev = 2
weight_mean = 1
weight_stddev = 0.1
lambda_param = 0.5
VSS = find_VSS(n, p, capacity_mean, capacity_stddev, weight_mean, weight_stddev, lambda_param, num_simulations)
print(f"The Value of Stochastic Solution (VSS) is {VSS:.2f}")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 601 rows, 600 columns and 1202 nonzeros
Model fingerprint: 0xe878e5bd
Variable types: 0 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-04, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 15.1698562
Presolve removed 601 rows and 600 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: -12.3219 15.1699 
No other solutions better than -12.3219

Optimal solution found (tolerance 1.00e-04)
Best objective -1.232188968768e+01, best bound -1.232188968768e+01, gap 0.0000%
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm]

In [57]:
# in_sample_stability_score = insample_stability(clusters,nodes, capacities, weights, lambda_param, num_runs=100)
# print("Insample Stability:",in_sample_stability_score)

In [58]:

# out_sample_stability_score = out_sample_stability(num_runs=100)
# print("Outsample Stability:",out_sample_stability_score)

In [59]:
greedy_clusters

[[162, 62, 58, 7, 104, 144, 185, 71, 77, 105],
 [172, 158, 120, 100, 45, 178, 118, 175, 155]]

In [60]:
clusters

[[0,
  3,
  5,
  8,
  12,
  13,
  16,
  20,
  21,
  22,
  27,
  29,
  32,
  33,
  34,
  36,
  45,
  50,
  53,
  57,
  64,
  66,
  69,
  74,
  76,
  79,
  80,
  82,
  84,
  92,
  95,
  101,
  103,
  108,
  111,
  117,
  122,
  123,
  127,
  130,
  131,
  144,
  147,
  150,
  152,
  156,
  165,
  171,
  174,
  175,
  177,
  178,
  180,
  186,
  187,
  190,
  191,
  196,
  197],
 [1,
  2,
  4,
  6,
  7,
  9,
  10,
  11,
  14,
  15,
  17,
  18,
  19,
  23,
  24,
  25,
  26,
  28,
  30,
  31,
  35,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  46,
  47,
  48,
  49,
  51,
  52,
  54,
  55,
  56,
  58,
  59,
  60,
  61,
  62,
  63,
  65,
  67,
  68,
  70,
  71,
  72,
  73,
  75,
  77,
  78,
  81,
  83,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  93,
  94,
  96,
  97,
  98,
  99,
  100,
  102,
  104,
  105,
  106,
  107,
  109,
  110,
  112,
  113,
  114,
  115,
  116,
  118,
  119,
  120,
  121,
  124,
  125,
  126,
  128,
  129,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  1